# Dataframe Construction
Combining features into a single dataframe to use for EDA.

In [3]:
import pandas as pd
from haversine import haversine

In [4]:
#Import the data
data1 = pd.read_csv('assets/train.csv')
df_train = pd.DataFrame(data1)

data2 = pd.read_csv('assets/spray.csv')
df_spray = pd.DataFrame(data2)

data3 = pd.read_csv('assets/weather.csv')
df_weather = pd.DataFrame(data3)

data4 = pd.read_csv('assets/test.csv')
df_test = pd.DataFrame(data4)

IOError: File assets/train.csv does not exist

In [82]:
#assign latitude longitude to each weather station.
df_weather['Latitude'] = df_weather['Station'].map({1:41.995, 2:41.786})
df_weather['Longitude'] = df_weather['Station'].map({1:-87.933, 2:-87.752})
df_weather.head(2)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Latitude,Longitude
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,M,0.0,0.00,29.10,29.82,1.7,27,9.2,41.995,-87.933
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,M,M,0.00,29.18,29.82,2.7,25,9.6,41.786,-87.752


# Train Data Set Station Assignment

In [83]:
#Nathan doing stupid stuff that shouldn't take so many lines of code.
df4 = df_weather.groupby(['Station']).mean().reset_index().transpose().reset_index()
df4.drop([0,1,2,3,4,5], inplace=True)
df4.drop(['index'], axis=1, inplace=True)
df4 = df4.reset_index()
df4.drop('index', axis=1, inplace=True)

In [84]:
#Function to find the distance from every trap to station 1 and station 0
df5_station = pd.DataFrame(columns=[df4.columns])
for i in df4.columns:
    HOLDER = []
    for x, y in enumerate(df_train['Latitude']):
        STATION = (df4[i].loc[0], df4[i].loc[1])
        TRAP = (y, df_train['Longitude'].loc[x])
        HOLDER.append(haversine(STATION, TRAP, miles=True))
    df5_station[i] = HOLDER

In [85]:
#Function to assign the closest station into a new variable and then add that variable as a column to df_train.
station = []
for i in df5_station.index:
    station.append(df5_station.loc[i].idxmin())
df_train['station'] = station

# Test Data Set Station Assignment

In [86]:
#Function to find the distance from every trap to station 1 and station 0
df6_station = pd.DataFrame(columns=[df4.columns])
for i in df4.columns:
    HOLDER = []
    for x, y in enumerate(df_test['Latitude']):
        STATION = (df4[i].loc[0], df4[i].loc[1])
        TRAP = (y, df_test['Longitude'].loc[x])
        HOLDER.append(haversine(STATION, TRAP, miles=True))
    df6_station[i] = HOLDER

In [87]:
#Function to assign the closest station into a new variable and then add that variable as a column to df_train.
station2 = []
for i in df6_station.index:
    station2.append(df6_station.loc[i].idxmin())
df_test['station'] = station2

# Assign Traps to Zip Code Data

In [89]:
df_test.columns

Index([u'Id', u'Date', u'Address', u'Species', u'Block', u'Street', u'Trap',
       u'AddressNumberAndStreet', u'Latitude', u'Longitude',
       u'AddressAccuracy', u'station'],
      dtype='object')

In [90]:
df_train.to_csv('assets/train_1.csv')

In [91]:
df_test.to_csv('assets/test_1.csv')

In [92]:
df_weather.to_csv('assets/weather_1.csv')